## Key Business Questions

#### Purpose: 

Analysis and Visualization of the data that provide key insights and help answer important business questions.


### Plot Sales Trends

Sales per Year / Quarter / Month / Week


### Plot Products with the highest Sales by Money, Volume


### Country-Wise Analysis:

Plot sales by country. Visualize the best selling product for each Country.


### Basket Size:

Average order value - What is the average $ amount per order.


### Customer Analysis:

Total Count of customers. Avg. Spend per customer. Avg Number of orders per customer. Distribution of customer spend. Distribution of customers by shopping frequency (number of customers that shop once, twice, three+ times).
